In [1]:
"""
Read VIIRS active fire data
"""

# Import packages
import os, time, glob
import pandas as pd
import geopandas as gpd
import rasterio as rio
import rioxarray as rxr
import h5py
import pyproj
import xarray as xr
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt
from affine import Affine
from osgeo import gdal, gdal_array, gdalconst, osr

# Projection information
geog = 'EPSG:4326'  # Geographic projection
prj = 'EPSG:5070'  # Projected coordinate system- WGS 84 NAD83 UTM Zone 13N

# File path information
maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/'

# File path information
print("Success")

Success


In [2]:
# Function to list files in a directory
def list_files(path, ext, recursive):
    """
    List files of a specific type in a directory or subdirectories
    """
    if recursive is True:
        return glob.glob(os.path.join(path, '**', '*{}'.format(ext)), recursive=True)
    else:
        return glob.glob(os.path.join(path, '*{}'.format(ext)), recursive=False)

# Get a list of .nc files
nc_files = list_files(os.path.join(maindir, 'data/spatial/raw/VIIRS'), "*.nc", recursive=True)
h5_files = list_files(os.path.join(maindir, 'data/spatial/raw/VIIRS'), "*.h5", recursive=True)

# Seperate into VNP14 and VJ114, VNP03
print(len(nc_files))

vnp = [f for f in nc_files if 'VNP14' in os.path.basename(f)] # VNP14IMG
vji = [f for f in nc_files if 'VJ114' in os.path.basename(f)] # VJI14IMG
geo = [f for f in h5_files if 'VNP03' in os.path.basename(f)] # VJI14IMG

print(f'VNP14IMG: {len(vnp)}')
print(f'VJI14IMG: {len(vji)}')
print(f'VNP03MODLL: {len(geo)}')


1750
VNP14IMG: 878
VJI14IMG: 872
VNP03MODLL: 0


In [3]:
# Split the path
parts = vnp[0].split("/")
for part in parts:
    if part.startswith("FIRED_"):
        fire_id = part
        break
print(fire_id)

FIRED_42260


In [4]:
vnp[0]

'/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/raw/VIIRS/FIRED_42260/VNP14IMG/VNP14IMG.A2018212.0942.002.2024081035625.nc'

In [5]:
# Library to work with netCDF files
from netCDF4 import Dataset

# Open a .nc file ("file_name")
ds = Dataset(vnp[0], 'r', format = 'NETCDF4')

print(ds)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    VNP02IMG: VNP02CCIMG.A2018212.0942.002.2022293171715.nc
    VNP03IMG: VNP03IMG.A2018212.0942.002.2021090055315.nc
    VNP02MOD: VNP02CCMOD.A2018212.0942.002.2022293171715.nc
    VNP02GDC: VNP02GDC.A2018212.0942.002.2021095211259.nc
    ProcessVersionNumber: 3.1.9
    ExecutableCreationDate: Oct 31 2023
    ExecutableCreationTime: 17:46:00
    SystemID: Linux minion20012 5.4.0-1093-fips #103-Ubuntu SMP Thu Feb 8 14:02:37 UTC 2024 x86_64
    Unagg_GRingLatitude: 18.343838,22.825336,43.746387,38.208107
    Unagg_GRingLongitude: -107.986687,-137.030869,-135.016525,-98.899147
    NorthBoundingCoordinate: 43.750687
    SouthBoundingCoordinate: 18.343838
    EastBoundingCoordinate: -98.89915
    WestBoundingCoordinate: -137.03087
    Satellite: NPP
    Day/Night/Both: Night
    FirePix: 1335
    DayPix: 0
    LandPix: 0
    NightPix: 41574400
    Conventions: CF-1.6
    WaterPix: 0
    MissingPix: 164681

In [6]:
day_night_flag = ds.getncattr('DayNightFlag')
print(day_night_flag)

Night


In [7]:
# Print a list of variables in file
list(ds.variables)

['FP_AdjCloud',
 'FP_AdjWater',
 'FP_MAD_DT',
 'FP_MAD_T4',
 'FP_MAD_T5',
 'FP_MeanDT',
 'FP_MeanRad13',
 'FP_MeanT4',
 'FP_MeanT5',
 'FP_Rad13',
 'FP_SolAzAng',
 'FP_SolZenAng',
 'FP_T4',
 'FP_T5',
 'FP_ViewAzAng',
 'FP_ViewZenAng',
 'FP_WinSize',
 'FP_confidence',
 'FP_day',
 'FP_latitude',
 'FP_line',
 'FP_longitude',
 'FP_power',
 'FP_sample',
 'algorithm QA',
 'fire mask']

NameError: name 'ds' is not defined

In [67]:
print(ds.variables[''])

<class 'netCDF4._netCDF4.Variable'>
uint32 algorithm QA(phony_dim_1, phony_dim_2)
    units: bit field
unlimited dimensions: 
current shape = (6496, 6400)
filling off


In [43]:
print(ds.variables['FP_confidence'][:])

[8 8 8 9 9 8 8 8 8 8 9 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 7 8 8 8 8 8 8 8 8 9 8
 8 8 8 8 7 8 8 9 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]


In [36]:
# Extract the acquisition date and time from the NetCDF file
start_time_str = ds.getncattr('PGE_StartTime')
acq_datetime = datetime.strptime(start_time_str, '%Y-%m-%d %H:%M:%S.%f')
# Calculate Julian Day
julian_day = acq_datetime.timetuple().tm_yday

print(f'Acquisition date/time: {acq_datetime}')
print(f'Acquisition Julian Day: {julian_day}')

Acquisition date/time: 2019-08-13 19:36:00
Acquisition Julian Day: 225


In [37]:
# Extract fire pixel information
latitudes = ds.variables['FP_latitude'][:]
longitudes = ds.variables['FP_longitude'][:]
fire_radiative_power = ds.variables['FP_power'][:]
fire_confidence = ds.variables['FP_confidence'][:]
day_night = ds.variables['FP_day'][:]

# Create a DataFrame with the fire pixel data
fire_data = pd.DataFrame({
    'latitude': latitudes,
    'longitude': longitudes,
    'fire_radiative_power': fire_radiative_power,
    'fire_confidence': fire_confidence,
    'day_night': day_night,
    'acq_datetime': acq_datetime,
    'acq_julian_day': julian_day
})

fire_data.head()

,latitude,longitude,fire_radiative_power,fire_confidence,day_night,acq_datetime,acq_julian_day
0,36.128933,-105.297821,5.851729,8,1,2019-08-13 19:36:00,225
1,35.017864,-111.902405,7.822381,8,1,2019-08-13 19:36:00,225
2,35.016457,-111.907646,7.822381,8,1,2019-08-13 19:36:00,225
3,35.015991,-111.905464,15.783789,9,1,2019-08-13 19:36:00,225
4,36.316563,-106.795013,18.076380,9,1,2019-08-13 19:36:00,225


In [26]:
print(len(fire_data))

63


In [27]:
import folium
from folium.plugins import MarkerCluster

# Create a Folium map centered around the mean latitude and longitude
m = folium.Map(location=[fire_data['latitude'].mean(), fire_data['longitude'].mean()], zoom_start=6)

# Add a marker cluster to the map
marker_cluster = MarkerCluster().add_to(m)

# Add markers to the map
for idx, row in fire_data.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f'FRP: {row["fire_radiative_power"]}',
        icon=folium.Icon(color='red')
    ).add_to(marker_cluster)

# Display the map
m.save('fire_radiative_power_map.html')
m